So sánh kết quả phương pháp và ground truth trên tập dữ liệu nhỏ (46 ảnh)

In [1]:
import os
import glob
import cv2
import numpy as np

In [2]:
def precision(TP: int, FP: int) -> float:
    epsilon = 1e-7
    return TP / (TP + FP + epsilon)

def recall(TP: int, FN: int) -> float:
    epsilon = 1e-7
    return TP / (TP + FN + epsilon)

def f1_score(precision: float, recall: float) -> float:
    if (precision + recall) == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

In [3]:
def calculate_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 0
    return iou

In [4]:
# Từ file txt thành masks
def txt_masks(file_path, size=640):
    masks = []
    if not os.path.exists(file_path):
        return masks

    with open(file_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5: continue

        coords = np.array([float(x) for x in parts[1:]])
        points = coords.reshape(-1, 2)

        points[:, 0] *= size
        points[:, 1] *= size
        points = points.astype(np.int32)

        mask = np.zeros((size, size), dtype=np.uint8)
        cv2.fillPoly(mask, [points], 1)
        masks.append(mask)

    return masks

In [5]:
# Tính kết quả từng ảnh
def evaluate_file(gt_path, pred_path, iou_thresh=0.5):
    gt_masks = txt_masks(gt_path)    
    pred_masks = txt_masks(pred_path) 

    tp = 0
    iou_sum = 0

    # Đánh dấu những mask đã được ghép cặp để không dùng lại
    gt_matched = [False] * len(gt_masks)
    pred_matched = [False] * len(pred_masks)

    # Tính tất cả các cặp IoU có thể xảy ra
    matches = []
    for i, gt_m in enumerate(gt_masks):
        for j, pred_m in enumerate(pred_masks):
            iou = calculate_iou(gt_m, pred_m)
            if iou >= iou_thresh:
                matches.append((iou, i, j))

    #  Sắp xếp giảm dần theo IoU
    matches.sort(key=lambda x: x[0], reverse=True)

    #  Duyệt và ghép cặp
    for iou, gt_idx, pred_idx in matches:
        # Nếu cả GT và Pred này đều chưa ai lấy
        if not gt_matched[gt_idx] and not pred_matched[pred_idx]:
            tp += 1
            iou_sum += iou
            # Đánh dấu là đã dùng
            gt_matched[gt_idx] = True
            pred_matched[pred_idx] = True

    n_gt = len(gt_masks)
    n_pred = len(pred_masks)

    fp = n_pred - tp 
    fn = n_gt - tp    
    
    miou = iou_sum / tp if tp > 0 else 0

    return miou, tp, fp, fn

In [6]:
# Tính tổng dữ liệu
def evaluate_dataset(gt_folder, pred_folder, iou_thresh=0.5):
    gt_files = glob.glob(os.path.join(gt_folder, "*.txt"))

    if len(gt_files) == 0:
        return

    total_tp = 0
    total_fp = 0
    total_fn = 0
    total_matched_iou = 0.0

    for gt_path in gt_files :
        filename = os.path.basename(gt_path)
        pred_path = os.path.join(pred_folder, filename)

        iou, tp, fp, fn = evaluate_file(gt_path, pred_path, iou_thresh)

        total_tp += tp
        total_fp += fp
        total_fn += fn
        total_matched_iou += iou

    # Precision:
    p = precision(total_tp,total_fp)

    # Recall:
    r = recall(total_tp,total_fn)

    # F1-Score:
    f1 = f1_score(p,r)

    # mIoU:
    m_iou = total_matched_iou / len(gt_files)

    print(f"IOU THRESHOLD = {iou_thresh}")
    #print(f"Tổng số file ảnh: {len(gt_files)}")
    #print(f"True Positives (TP):   {total_tp} ")
    #print(f"False Positives (FP):  {total_fp} ")
    #print(f"False Negatives (FN):  {total_fn} ")
    print(f" Precision: {p:.4f}")
    print(f" Recall: {r:.4f}")
    print(f" F1-Score: {f1:.4f}")
    print(f" m_IoU:{m_iou:.4f} ")

In [7]:
GT_DIR = '/home/tdh/test/gt/'
PRED_DIR = '/home/tdh/test/q_tiny'

for i in range(5,10) :
    evaluate_dataset(GT_DIR, PRED_DIR, iou_thresh=i/10)

IOU THRESHOLD = 0.5
 Precision: 0.8982
 Recall: 0.8434
 F1-Score: 0.8699
 m_IoU:0.8675 
IOU THRESHOLD = 0.6
 Precision: 0.8797
 Recall: 0.8261
 F1-Score: 0.8520
 m_IoU:0.8747 
IOU THRESHOLD = 0.7
 Precision: 0.8507
 Recall: 0.7988
 F1-Score: 0.8240
 m_IoU:0.8829 
IOU THRESHOLD = 0.8
 Precision: 0.7577
 Recall: 0.7115
 F1-Score: 0.7338
 m_IoU:0.9010 
IOU THRESHOLD = 0.9
 Precision: 0.4504
 Recall: 0.4229
 F1-Score: 0.4362
 m_IoU:0.9331 


In [8]:
GT_DIR = '/home/tdh/test/gt'
PRED_DIR = '/home/tdh/test/q_large'

for i in range(5,10) :
    evaluate_dataset(GT_DIR, PRED_DIR, iou_thresh=i/10)

IOU THRESHOLD = 0.5
 Precision: 0.9039
 Recall: 0.8450
 F1-Score: 0.8735
 m_IoU:0.8719 
IOU THRESHOLD = 0.6
 Precision: 0.8871
 Recall: 0.8293
 F1-Score: 0.8573
 m_IoU:0.8782 
IOU THRESHOLD = 0.7
 Precision: 0.8545
 Recall: 0.7988
 F1-Score: 0.8257
 m_IoU:0.8876 
IOU THRESHOLD = 0.8
 Precision: 0.7734
 Recall: 0.7230
 F1-Score: 0.7473
 m_IoU:0.9029 
IOU THRESHOLD = 0.9
 Precision: 0.4665
 Recall: 0.4361
 F1-Score: 0.4508
 m_IoU:0.9351 


In [9]:
GT_DIR = '/home/tdh/test/gt/'
PRED_DIR = '/home/tdh/test/tiny'

for i in range(5,10) :
    evaluate_dataset(GT_DIR, PRED_DIR, iou_thresh=i/10)

IOU THRESHOLD = 0.5
 Precision: 0.9003
 Recall: 0.8409
 F1-Score: 0.8696
 m_IoU:0.8680 
IOU THRESHOLD = 0.6
 Precision: 0.8808
 Recall: 0.8228
 F1-Score: 0.8508
 m_IoU:0.8754 
IOU THRESHOLD = 0.7
 Precision: 0.8482
 Recall: 0.7923
 F1-Score: 0.8193
 m_IoU:0.8850 
IOU THRESHOLD = 0.8
 Precision: 0.7564
 Recall: 0.7065
 F1-Score: 0.7306
 m_IoU:0.9023 
IOU THRESHOLD = 0.9
 Precision: 0.4492
 Recall: 0.4196
 F1-Score: 0.4339
 m_IoU:0.9339 


In [10]:
GT_DIR = '/home/tdh/test/gt'
PRED_DIR = '/home/tdh/test/large'

for i in range(5,10) :
    evaluate_dataset(GT_DIR, PRED_DIR, iou_thresh=i/10)

IOU THRESHOLD = 0.5
 Precision: 0.9072
 Recall: 0.8467
 F1-Score: 0.8759
 m_IoU:0.8707 
IOU THRESHOLD = 0.6
 Precision: 0.8905
 Recall: 0.8310
 F1-Score: 0.8597
 m_IoU:0.8770 
IOU THRESHOLD = 0.7
 Precision: 0.8542
 Recall: 0.7972
 F1-Score: 0.8247
 m_IoU:0.8876 
IOU THRESHOLD = 0.8
 Precision: 0.7668
 Recall: 0.7156
 F1-Score: 0.7403
 m_IoU:0.9043 
IOU THRESHOLD = 0.9
 Precision: 0.4753
 Recall: 0.4435
 F1-Score: 0.4588
 m_IoU:0.9350 
